In [1]:
!pip install accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

In [2]:
import os
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

import torch
import random
import numpy as np
from datasets import load_dataset
from transformers import LlamaTokenizerFast, LlamaForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig
from trl import SFTTrainer

In [3]:
BASE_MODEL="TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
DATASET_NAME="tatsu-lab/alpaca"
RANDOM_SEED = 42
BATCH_SIZE = 2
EPOCHS = 2
LEARNING_RATE = 2e-5
LR_SCHEDULER = "cosine"
OPTIMIZER = "paged_adamw_32bit"
BETA1 = 0.9
BETA2 = 0.95
EPSILON = 1e-5
WARMUP_STEPS = 2000
LOGGING_STEPS = 1000
WEIGHT_DECAY = 0.1
MAX_SEQUENCE_LENGTH = 2048
FP16 = True
GRADIENT_CLIPPING = 1

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
tokenizer = LlamaTokenizerFast.from_pretrained(BASE_MODEL)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
dataset = load_dataset(DATASET_NAME, split="train[:10%]")
print(len(dataset))

Generating train split: 0 examples [00:00, ? examples/s]

5200


In [6]:
print(dataset[0])
print(dataset[0]["text"])
print("-"*30)
print(dataset[6])
print(dataset[6]["text"])

{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong.

In [7]:
local_dataset = load_dataset("json", data_files="/content/drive/MyDrive/llama/dummy-data.json")["train"]
print(len(local_dataset))

Generating train split: 0 examples [00:00, ? examples/s]

2


In [8]:
print(local_dataset[0])
print("-"*30)
print(local_dataset[1])

{'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}
------------------------------
{'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nExplain why the following fraction is equivalent to 1/4\n\n### Input:\n4/16\n\n### Response:\nThe fraction 4/16 is equivalent to 1/4 because both numerators and denominators are divisible by 4. Dividing both the top and bottom numbers by 4 yields the fraction 1/4.'}


In [9]:
def add_eos_token(inp):
    inp["text"] = inp["text"] + tokenizer.eos_token
    return(inp)

updated_dataset = dataset.map(add_eos_token)
print(updated_dataset[0]["text"])

Map:   0%|          | 0/5200 [00:00<?, ? examples/s]

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.</s>


In [10]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [11]:
model = LlamaForCausalLM.from_pretrained(BASE_MODEL, quantization_config=quant_config)

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [12]:
peft_params = LoraConfig(
    r=64,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [13]:
training_params = TrainingArguments(
    output_dir="./deleteme",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    optim=OPTIMIZER,
    logging_steps=LOGGING_STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    fp16=FP16,
    warmup_steps=WARMUP_STEPS,
    lr_scheduler_type=LR_SCHEDULER,
    adam_beta1=BETA1,
    adam_beta2=BETA2,
    adam_epsilon=EPSILON,
    max_grad_norm=GRADIENT_CLIPPING
)

In [14]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    peft_config=peft_params,
    tokenizer=tokenizer,
    args=training_params,
    max_seq_length=MAX_SEQUENCE_LENGTH
)

Map:   0%|          | 0/5200 [00:00<?, ? examples/s]

In [15]:
trainer.train()

Step,Training Loss
1000,1.740000
2000,1.052200
3000,0.960900
4000,0.966100
5000,0.964300


TrainOutput(global_step=5200, training_loss=1.1305534509512094, metrics={'train_runtime': 1466.273, 'train_samples_per_second': 7.093, 'train_steps_per_second': 3.546, 'total_flos': 1.0143763006611456e+16, 'train_loss': 1.1305534509512094, 'epoch': 2.0})

In [16]:
prompt = "Who is Napoleon Bonaparte?"
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512,
                num_beams=10,
                early_stopping=True,
                no_repeat_ngram_size=2)

prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{prompt}

### Response:
"""

result = pipe(prompt)
print(result[0]['generated_text'])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Who is Napoleon Bonaparte?

### Response:
<NAME> (1759-1815) was a French military and political leader who is best known for his role in the French Revolution and the Napoleonic Wars. He is considered to be one of the most influential figures in history, and his legacy is still felt today. Napoleon was born into a wealthy family in France and began his military career at the age of 14. After serving in several campaigns, he rose to prominence as a military leader and eventually became the Emperor of France. His rule was marked by a series of wars against other European powers, including the Austrian Empire, Prussia, the Kingdom of Sardinia and Austria-Hungary. Despite his successes, Napoleon's reign was marred by corruption and abuse of power, which led to his downfall and eventual exile to the island of St. Helena off the coast of Africa. Following his death, hi

In [17]:
prompt = "What is your favorite color?"
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512,
                num_beams=10,
                early_stopping=True,
                no_repeat_ngram_size=2)

prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{prompt}

### Response:
"""

result = pipe(prompt)
print(result[0]['generated_text'])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is your favorite color?

### Response:
I love all colors, but my favorite is probably blue. It's soothing and calming, and it makes me feel like I'm in a good mood. I also love pink and purple, which are feminine and romantic. Green is also a great color for me, as it reminds me of nature and the outdoors. White is a neutral color that can be worn with any outfit, making it a versatile choice for any occasion. Finally, red is the color of passion and energy, so it is perfect for someone who is passionate about life and wants to make a positive impact on the world. All of these colors have their own unique qualities that make them appealing to different people. Ultimately, it all comes down to personal preference and how you want to express yourself through your wardrobe. No matter what color you choose, you'll be sure to stand out from the crowd!



In [18]:
prompt = "Which of the following cities is the capital of Spain?"
context = "Istanbul, Madrid, New York, London"
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512,
                num_beams=10,
                early_stopping=True,
                no_repeat_ngram_size=2)


prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{prompt}

### Input:
{context}

### Response:
"""

result = pipe(prompt)
print(result[0]['generated_text'])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Which of the following cities is the capital of Spain?

### Input:
Istanbul, Madrid, New York, London

### Response:
Madrid. Madrid is Spain's capital city and the largest city in the country, with a population of over 3.6 million people. The city is known for its rich history and culture, as well as its thriving economy and vibrant nightlife. It is also home to several important landmarks, such as the Prado Museum, the Royal Palace, and Plaza de Oriente, which is a popular tourist destination. In addition to its cultural and historical attractions, it also offers a wide range of shopping and dining options, making it an attractive destination for tourists and locals alike. 

